In [1]:
const _ = require('lodash')
const traverse = require('traverse')
const sparqljs = require('sparqljs')

const eachFileInDir = require('./eachFileInDir')
const countInQueries = require('./countInQueries')
const namespaces = require('./namespaces')
const parser = new sparqljs.Parser(namespaces)
const Query = require('./Query')

const WIKI = 'data/wikidatawiki'

## Valid Queries

In [4]:
var isValidQuery = (query) => {
    return true // will internally throw an exception when parsed
}

countInQueries(WIKI, isValidQuery)

data/wikidatawiki 7370 7370


## OPTIONAL

In [18]:
var hasOptional = (query) => {
    return query.traverse().reduce(function(acc, node) {
        return acc || this.notLeaf && node.type === 'optional'
    }, false)
}
countInQueries(WIKI, hasOptional)

data/wikidatawiki 7370 2297


## FILTER

In [20]:
var hasFilter = (query) => {
    return query.traverse().reduce(function(acc, node) {
        return acc || this.notLeaf && node.type === 'filter'
    }, false)
}
countInQueries(WIKI, hasFilter)

data/wikidatawiki 7370 1622


## ORDER BY

In [21]:
var hasOrderBy = (query) => {
    return query.getParsed().order
}
countInQueries(WIKI, hasOrderBy)

data/wikidatawiki 7370 2040


## DISTINCT

In [8]:
var hasDistinct = (query) => {
    return query.getParsed().distinct
}
countInQueries(WIKI, hasDistinct)

data/wikidatawiki 7370 2017


## GROUP BY

In [27]:
var hasGroupBy = (query) => {
    return query.getParsed().group
}
countInQueries(WIKI, hasGroupBy)

data/wikidatawiki 7370 679


## VALUES

In [23]:
var hasValues = (query) => {
    return query.traverse().reduce(function(acc, node) {
        return acc || this.notLeaf && node.type === 'values'
    }, false)
}
countInQueries(WIKI, hasValues)

data/wikidatawiki 7370 543


## UNION

In [25]:
var hasUnion = (query) => {
    return query.traverse().reduce(function(acc, node) {
        return acc || this.notLeaf && node.type === 'union'
    }, false)
}
countInQueries(WIKI, hasUnion)

data/wikidatawiki 7370 1079


## NOT EXIST

In [12]:
var hasNotExist = (query) => {
    return query.traverse().reduce(function(acc, node) {
        return acc || this.notLeaf && node.operator === 'notexists'
    }, false)
}
countInQueries(WIKI, hasNotExist)

data/wikidatawiki 7370 192


## BIND

In [14]:
var hasBind = (query) => {
    return query.traverse().reduce(function(acc, node) {
        return acc || this.notLeaf && node.type === 'bind'
    }, false)
}
countInQueries(WIKI, hasBind)

data/wikidatawiki 7370 1283


## MINUS

In [2]:
var hasMinus = (query) => {
    return query.traverse().reduce(function(acc, node) {
        return acc || this.notLeaf && node.type === 'minus'
    }, false)
}
countInQueries(WIKI, hasMinus)

data/wikidatawiki 7370 1604


## Subqueries

In [3]:
var hasSubQuery = (query) => {
    return query.traverse().reduce(function(acc, node) {
        return acc || this.level > 0 && this.notLeaf && node.type === 'query'
    }, false)
}
countInQueries(WIKI, hasSubQuery)

data/wikidatawiki 7370 160


## Multiple Subject Queries

In [4]:
var hasMultipleSubjects = (query) => {
    const objects = query.getObjects()
    
    return query.getSubjects().some((subject) => {
        return typeof subject === 'string' && subject.startsWith('?') && objects.includes(subject)
    })
}
countInQueries(WIKI, hasMultipleSubjects)

data/wikidatawiki 7349 2664


## Property Path

In [5]:
var hasPropertyPath = (query) => {
    return query.getPredicates().some(predicate => predicate.type === 'path')
}
countInQueries(WIKI, hasPropertyPath)

data/wikidatawiki 7349 2350


## Most Common Property Path Predicates

In [6]:
var predicates = []
eachFileInDir(WIKI, (query, resolve) => {
    var query = new Query(query, parser.parse(query))
    
    predicates = traverse(query.getPredicates().filter(predicate => predicate.type === 'path')).reduce((acc, node) => {
        if (typeof node === 'string' && node.includes('prop/direct/P')) {
            acc.push(node)
        }
        
        return acc
    }, predicates)
    
    resolve()
}).then(() => {
    console.log(_.countBy(predicates, _.identity))
})

Promise { <pending> }

{ 'http://www.wikidata.org/prop/direct/P131': 735,
  'http://www.wikidata.org/prop/direct/P31': 1364,
  'http://www.wikidata.org/prop/direct/P279': 2080,
  'http://www.wikidata.org/prop/direct/P1181': 5,
  'http://www.wikidata.org/prop/direct/P361': 33,
  'http://www.wikidata.org/prop/direct/P106': 90,
  'http://www.wikidata.org/prop/direct/P2596': 1,
  'http://www.wikidata.org/prop/direct/P40': 41,
  'http://www.wikidata.org/prop/direct/P17': 17,
  'http://www.wikidata.org/prop/direct/P298': 1,
  'http://www.wikidata.org/prop/direct/P462': 5,
  'http://www.wikidata.org/prop/direct/P172': 1,
  'http://www.wikidata.org/prop/direct/P91': 1,
  'http://www.wikidata.org/prop/direct/P1269': 3,
  'http://www.wikidata.org/prop/direct/P1435': 2,
  'http://www.wikidata.org/prop/direct/P276': 13,
  'http://www.wikidata.org/prop/direct/P16': 2,
  'http://www.wikidata.org/prop/direct/P4224': 1,
  'http://www.wikidata.org/prop/direct/P921': 6,
  'http://www.wikidata.org/prop/direct/P20': 9,
  'http:

## Object Variable Referencing (TODO: how is this really called?)

In [7]:
// This is about queries with object variables that are references as objects in another triple.
// This functionality is needed for cyclic links to answer questions like "What movies have actors starring together with their children?

var hasObjectReference = (query) => {
    const objectCounts = _.countBy(query.getObjects(), _.identity)
    return _.size(_.pickBy(objectCounts, (count, obj) => count > 1 && _.startsWith(obj, '?'))) > 0
}
countInQueries(WIKI, hasObjectReference)

data/wikidatawiki 7349 625


## Relevant Object Variable References

In [8]:
var hasRelevantObjectReference = (query) => {
    const objectCounts = _.countBy(query.getObjects(), _.identity)
    const referencedObjects = _.pickBy(objectCounts, (count, obj) => count > 1 && _.startsWith(obj, '?'))
    const predicatesWithObj = query.traverse().reduce(function(acc, node) {

      if (typeof node == 'object' && node.object && referencedObjects[node.object]) acc.push(node.predicate)

      return acc
    }, [])
    const relevantPredicatesWithObj = predicatesWithObj.filter(predicate => {
        return traverse(predicate).reduce((acc, node) => {
            return acc || typeof node === 'string' && node.includes('http://www.wikidata.org/prop/direct/')
        }, false)
    })
    
    
    return _.size(relevantPredicatesWithObj) > 1
}
countInQueries(WIKI, hasRelevantObjectReference)

data/wikidatawiki 7349 435


## Cycles

In [9]:
var transitive = (graph) => { // silly
    do {
        var change = false
        
        for (var subject in graph) {
            var subjectReachability = graph[subject]
            for (var i = 0; i < graph[subject].length; i++) {
                if (graph[graph[subject][i]]) {
                    subjectReachability = subjectReachability.concat(graph[graph[subject][i]])
                }
            }
            if (_.uniq(subjectReachability).length > graph[subject].length) {
                change = true
                graph[subject] = _.uniq(subjectReachability)
            }
        }
    } while (change)
        
    return graph
}

var hasCycles = (graph) => {
    for (var subject in graph) {
        if (graph[subject].includes(subject)) return true
    }
    return false
}

var hasCycle = (query) => {
    var reachability = query.traverse().reduce((acc, node) => {
        if (node.subject && node.object.startsWith('?')) {
            if (!acc[node.subject]) acc[node.subject] = []
            acc[node.subject].push(node.object)
        }
        
        return acc
    }, {})

    return hasCycles(transitive(reachability))
}

countInQueries(WIKI, hasCycle)

data/wikidatawiki 7349 58


## wdt:P31/wdt:P279*

In [11]:
var hasInstanceOfSubclassOf = (query) => {
    return query.getPredicates().some(predicate => {
        return ( predicate.type === 'path'
            && predicate.pathType === '/'
            && predicate.items[0] === 'http://www.wikidata.org/prop/direct/P31'
            && predicate.items[1].pathType === '*'
            && predicate.items[1].items[0] === 'http://www.wikidata.org/prop/direct/P279' )
    })
}
countInQueries(WIKI, hasInstanceOfSubclassOf)

data/wikidatawiki 7349 1060


## wdt:P279*

In [12]:
var hasSubclassOf = function hasSubclassOf(predicate) {
    if (predicate.type !== 'path') return false
    if (predicate.pathType === '*' && predicate.items.includes('http://www.wikidata.org/prop/direct/P279')) {
        return true
    }
    
    return predicate.items.reduce((any, predicate) => { return any || hasSubclassOf(predicate) }, false)
}
var hasInstanceOfSubclassOf = (query) => {
    return query.getPredicates().some(hasSubclassOf)
}
countInQueries(WIKI, hasInstanceOfSubclassOf)

data/wikidatawiki 7349 1381


## Use of Qualifiers

In [13]:
var hasQualifiers = (query) => {
    return query.getRaw().includes('pq:P')
}

countInQueries(WIKI, hasQualifiers)

data/wikidatawiki 7370 1198


## Use of References

In [14]:
var hasReferences = (query) => {
    return query.getRaw().includes('pr:P')
}

countInQueries(WIKI, hasReferences)

data/wikidatawiki 7370 32


## Sitelinks

In [15]:
var hasSitelinks = (query) => {
    return query.getRaw().includes('schema:about')
}

countInQueries(WIKI, hasSitelinks)

data/wikidatawiki 7370 575


In [16]:
var predicates = {}

eachFileInDir(WIKI, (query, resolve) => {
    var wikibasePredicates = query.match(/wikibase:\w+/g)
    if (wikibasePredicates) wikibasePredicates.forEach((predicate) => {
        if (predicates[predicate]) predicates[predicate]++
        else predicates[predicate] = 1
    })
    resolve()
}).then(() => {
    var tuples = []
    for (var predicate in predicates) tuples.push([predicate, predicates[predicate]])
    console.log(tuples.sort((a, b) => b[1] - a[1]).slice(0, 5))
})

Promise { <pending> }

[ [ 'wikibase:label', 1267 ],
  [ 'wikibase:language', 1267 ],
  [ 'wikibase:sitelinks', 275 ],
  [ 'wikibase:statements', 210 ],
  [ 'wikibase:directClaim', 71 ] ]


## Features per Query

In [32]:
var allFeatures = [
    hasOptional, hasFilter, hasOrderBy,
    hasValues, hasUnion, hasMinus,
    hasSubQuery, hasMultipleSubjects, hasPropertyPath,
    hasSubclassOf, hasQualifiers, hasReferences,
    hasGroupBy
]
var featuresPerQuery = (new Array(12)).fill(0)

eachFileInDir(WIKI, (query, resolve) => {
    let features = 0
    allFeatures.forEach((hasFeature) => {
        if (hasFeature(new Query(query, parser.parse(query)))) features++
    })
    featuresPerQuery[features]++
    resolve()
}).then(() => {
    console.log(featuresPerQuery)
})

// EN: [ 432, 111, 248, 117, 71, 9, 3, 1, 0, 0, 0, 0 ]
// DE: [ 85,  451,  74,  40,  7, 3, 0, 9, 0, 0, 0, 0 ]

// Updated
// EN: [ 432, 101, 222, 103, 79, 40, 9, 4, 2, 0, 0, 0 ]
// DE: [ 84, 343, 170, 27, 26, 7, 3, 9, 0, 0, 0, 0 ]

Promise { <pending> }

[ 1185, 2410, 958, 957, 716, 815, 158, 93, 18, 38, 1, 0 ]


## Finding a subset of features that is sufficient for most queries

In [31]:
var allFeatures = [
    hasOptional, hasFilter, hasOrderBy,
    hasValues, hasUnion, hasMinus,
    hasSubQuery, 
    hasGroupBy, hasPropertyPath
]
var subset = [
    hasOptional, hasFilter,
    hasValues, hasOrderBy, hasPropertyPath
];
var queriesProduceable = 0

var isProduceableWithSubset = (query) => {
    let featuresUsed = 0
    let subsetFeaturesUsed = 0

    allFeatures.forEach((hasFeature) => {
        if (hasFeature(query)) featuresUsed++
    })
    subset.forEach((hasFeature) => {
        if (hasFeature(query)) subsetFeaturesUsed++
    })
    
    return featuresUsed === subsetFeaturesUsed
}

countInQueries(WIKI, isProduceableWithSubset)

[ 1185, 2410, 958, 958, 716, 815, 158, 93, 18, 38, 1, 0 ]
[ 1411, 2975, 1121, 1270, 901, 869, 170, 96, 18, 38, 2, 0 ]
data/wikidatawiki 7349 4337


## Queries with Wikidata-URI subjects (not variables)

In [30]:
var hasURISubject = (query) => {
    return query.getSubjects().some(subject => subject.startsWith('http://www.wikidata.org/entity/Q'))
}

countInQueries(WIKI, hasURISubject)

data/sparql_queries_dewiki 669 0
data/sparql_queries_enwiki 992 2
